# How to use the API to pickle data

In [1]:
import os
import sys
from datetime import date
from pathlib import Path

curr_dir = Path(os.path.split(os.getcwd())[0])
metadata_dir = str(curr_dir)
if metadata_dir not in sys.path:
    sys.path.insert(0, metadata_dir)

import os
import glob
import time
import datetime
import numpy as np
import compress_pickle
import pandas as pd
from functools import partial
from multiprocessing import Pool
from bisect import bisect_left
from scipy.interpolate import interp1d
from scipy import signal
import matplotlib.pyplot as plt
from nptdms import TdmsFile 

## define source_path and destination_path

In [2]:
print("current working directory:",os.getcwd())

if os.path.isdir("/eos/project/m/ml-for-alarm-system/private/"):
    access2EOS = True
    print("Access to EOS possible")
    root_directory = "/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/"
else:
    access2EOS = False
    print("Access to EOS IMpossible")
    # search for a list of root_directory depending on the machine used to execute the notebook
    for directory in ["/home/thomas/cernboxML/private/CLIC_data_transfert/",
                      "?Christoph/CLIC_data_transfert/",
                      "?Andrea/CLIC_data_transfert/"]:
        if os.path.isdir(directory):
            root_directory = directory
print("Root directory:",root_directory)
 
# one week    
#source_path = root_directory + "PSI1_XB2_Data/"

# 6 months
source_path = root_directory + "CLIC_DATA_Xbox3/"

# don't forget the last / in the path
destination_path = source_path[0:-1]+"___pickle/" 

print("source_path of the data:",source_path)
print("destination_path of the pickle:",destination_path)

current working directory: /eos/home-c/cobermai/SWAN_projects/clicml/clic_xbox3/notebooks
Access to EOS possible
Root directory: /eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/
source_path of the data: /eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/CLIC_DATA_Xbox3/
destination_path of the pickle: /eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/CLIC_DATA_Xbox3___pickle/


## Trend data

In [91]:
listOfTrendDataFiles = glob.glob(source_path+"Trend*.tdms")
len(listOfTrendDataFiles)

38

In [118]:
listOfTrendDataFiles[0]

'/eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/CLIC_DATA_Xbox3/TrendData_20171118.tdms'

In [137]:
import gc
i =1
firstFile = True
for dataFile in listOfTrendDataFiles:
    print('{}/{} File: {}'.format(i,len(listOfTrendDataFiles),dataFile))
    TrendData = TdmsFile.read(dataFile)

    for group in  TrendData.groups():
        group_name = group.name

        df_Trend_New = TrendData[group_name].as_dataframe()
        df_Trend_New[group_name] = group_name
        
        if firstFile:
            df_Trend = df_Trend_New
            firstFile = False
        else:
            df_Trend = df_Trend.append(df_Trend_New, ignore_index=True, sort=False)

        print('New File: {}  Total: {}'.format(len(df_Trend_New),len(df_Trend)))
        
    i = i+1
    # Free memory
    del TrendData
    gc.collect()

df_Trend.to_pickle(destination_path+"XBOX_3_Trenddata.p.gz") 

# Event data

In [4]:
listOfEventDataFiles = glob.glob(source_path+"Event*.tdms")
len(listOfEventDataFiles)

58

In [5]:
import gc

firstFile = True
i = 1
for dataFile in listOfEventDataFiles:
    
    print('{}/{} File: {}'.format(i,len(listOfEventDataFiles),dataFile))
    EventData = TdmsFile.read(dataFile)

    for group in  EventData.groups():
        group_name = group.name
        df_event = EventData[group_name].as_dataframe()
           
        df_Eventproperties_New = pd.DataFrame(dict(group.properties), columns=group.properties.keys(), index=[0])
                
        if firstFile:
            df_Eventproperties = df_Eventproperties_New
            firstFile = False
        else:
            df_Eventproperties = df_Eventproperties.append(df_Eventproperties_New, ignore_index=True, sort=False)
        break
    i = i+1
    # Free memory
    del EventData
    gc.collect()
    break

#df_Eventproperties.to_pickle("XBOX_3_Eventproperties.p.gz") 

1/58 File: /eos/project/m/ml-for-alarm-system/private/CLIC_data_transfert/CLIC_DATA_Xbox3/EventDataB_20171118.tdms


In [6]:
df_event

,PKRA,PLRA,PERA,DC_UP,PKRB,PLRB,PERB,DC_DOWN,KLYINA_amp,KLYINA_ph,...,Reference228B_amp,Reference228B_ph,PSI_amp,PSI_ph,PSR_amp,PSR_ph,PEI_amp,PEI_ph,Reference1600_amp,Reference1600_ph
0,1.0,-8017.0,-6971.0,-84.0,-7084.0,-5813.0,-5248.0,84.0,16.0,-9092.0,...,3472.0,12934.0,7,0,42,15335,26,-249,5527,11328
1,32703.0,-7823.0,-6994.0,-80.0,-7249.0,-5807.0,-5345.0,95.0,16.0,-9092.0,...,3470.0,12929.0,8,0,35,8915,15,0,5516,11330
2,-7648.0,-7970.0,-7156.0,-81.0,-7225.0,-5905.0,-5297.0,85.0,15.0,-8596.0,...,3470.0,12932.0,16,5239,32,723,4,0,5505,11325
3,-7608.0,-7903.0,-6973.0,-86.0,-7150.0,-5898.0,-5119.0,85.0,17.0,-10500.0,...,3471.0,12911.0,29,8595,23,-8115,15,0,5506,11338
4,-7563.0,-7839.0,-6822.0,-83.0,-7258.0,-5983.0,-5280.0,89.0,12.0,-8590.0,...,3476.0,12916.0,28,10498,41,-15336,13,0,5519,11333
5,-7580.0,-8173.0,-7118.0,-83.0,-7315.0,-5742.0,-5308.0,88.0,20.0,-5886.0,...,3483.0,12919.0,10,0,15,0,9,0,5516,11322
6,-7609.0,-7825.0,-6951.0,-85.0,-7110.0,-5843.0,-5215.0,86.0,10.0,-2307.0,...,3492.0,12942.0,10,0,34,723,16,-14729,5515,11314
7,-7554.0,-7944.0,-7284.0,-90.0,-7253.0,-5930.0,-5232.0,93.0,1.0,-7293.0,...,3480.0,12932.0,17,5233,17,-7795,10,0,5522,11332
8,-7770.0,-7867.0,-6895.0,-86.0,-7183.0,-5911.0,-5177.0,82.0,21.0,-13106.0,...,3468.0,12911.0,13,0,44,15009,14,0,5543,11331
9,-7691.0,-8017.0,-6950.0,-83.0,-7266.0,-5937.0,-5157.0,84.0,32.0,-11145.0,...,3480.0,12871.0,10,0,24,10499,13,0,5543,11317


In [132]:
df_event.columns.values

array(['PKRA', 'PLRA', 'PERA', 'DC_UP', 'PKRB', 'PLRB', 'PERB', 'DC_DOWN',
       'KLYINA_amp', 'KLYINA_ph', 'PKIA_amp', 'PKIA_ph', 'PLIA_amp',
       'PLIA_ph', 'Reference228A_amp', 'Reference228A_ph', 'KLYINB_amp',
       'KLYINB_ph', 'PKIB_amp', 'PKIB_ph', 'PLIB_amp', 'PLIB_ph',
       'Reference228B_amp', 'Reference228B_ph', 'PSI_amp', 'PSI_ph',
       'PSR_amp', 'PSR_ph', 'PEI_amp', 'PEI_ph', 'Reference1600_amp',
       'Reference1600_ph'], dtype=object)